# Labelling of cars

We label cars by locating 'car-shaped' clusters above roads and parking bays.

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
# Import modules.
import numpy as np
import time
import logging

import upcp.utils.log_utils as log_utils
import upcp.utils.las_utils as las_utils
import upcp.utils.csv_utils as csv_utils
import upcp.utils.ahn_utils as ahn_utils
import upcp.utils.bgt_utils as bgt_utils
import upcp.fusion as fusion
from upcp.labels import Labels
from upcp.pipeline import Pipeline

# INFO messages will be printed to console.
log_utils.reset_logger()
log_utils.add_console_logger(level=logging.DEBUG)

## Car fuser based on cluster dimensions and BGT information
First, download and parse the required BGT data in notebook [1. Generate reference data](1.%20Generate%20reference%20data.ipynb).

In [ ]:
# The already labelled ground and building points must be removed.
exclude_labels = (Labels.GROUND, Labels.ROAD, Labels.BUILDING)

In [ ]:
# Select the file to process.
tilecode = '2386_9702'

in_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'
out_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'

# Data folder for the fusers.
ahn_data_folder = '../datasets/ahn/'
pc_data_folder = '../datasets/pointcloud/'
bgt_data_file = '../datasets/bgt/bgt_roads_demo.csv'

# We need elevation data to determine object height above ground.
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

# Create reader for BGT road part polygons.
bgt_road_reader = bgt_utils.BGTPolyReader(bgt_file=bgt_data_file)

In [ ]:
# To find appropriate dimension ranges, see e.g. https://nl.automobiledimension.com/.
car_params = {'min_height': 1.2, 'max_height': 2.4, 'min_width': 1.4, 'max_width': 2.4, 'min_length': 3.0, 'max_length': 6.0}

# Car fuser using a clustering algorithm and BGT road data.
car_fuser = fusion.CarFuser(Labels.CAR, ahn_reader, bgt_reader=bgt_road_reader, grid_size=0.05,
                            min_component_size=5000, params=car_params)

## Process point clouds and save the results
The pipeline can now be used to process point clouds. Labels will be written to the LAS file as a new extended field called label.

In [ ]:
# Set-up the order in how we want to label.
process_sequence = (car_fuser,)
pipeline = Pipeline(processors=process_sequence, exclude_labels=exclude_labels, caching=False)

## Process a single file

In [ ]:
# Process the file.
pipeline.process_file(in_file, out_file=out_file)